In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive

import json

# Load the JSON data from the file
with open('./data/points_map_2024.json', 'r') as f:
    data = json.load(f)

# Convert the dictionary to a DataFrame and name the columns appropriately
socring_curve_24 = pd.DataFrame(list(data.items()), columns=['place', 'points'])

# Optionally, convert the columns to numeric types
socring_curve_24['place'] = socring_curve_24['place'].astype(int)
socring_curve_24['points'] = socring_curve_24['points'].astype(int)

# Sort by place if needed
socring_curve_24.sort_values('place', inplace=True)
socring_curve_24.reset_index(drop=True, inplace=True)

# If using Jupyter Notebook, uncomment the line below for inline plots
# %matplotlib inline

def exponential_smoothing(data, alpha, start_value):
    """
    Applies single exponential smoothing to a time series.

    Parameters:
    -----------
    data : array-like
        The data points to smooth (first element = 1, rest = 0).
    alpha : float
        The smoothing factor (0 <= alpha <= 1).
    start_value : float
        The initial value used for the smoothed series (overrides the first data point).

    Returns:
    --------
    smoothed_data : list
        The exponentially smoothed data representing the fantasy league scoring curve.
    """
    # Set the first smoothed point to start_value
    smoothed_data = [start_value]
    
    # Subsequent points follow single exponential smoothing
    for i in range(1, len(data)):
        smoothed_value = alpha * data[i] + (1 - alpha) * smoothed_data[i - 1]
        smoothed_data.append(smoothed_value)
    
    return smoothed_data

# Create a data array for 30 positions:
# The first entry is 1 (representing 1st place), and the rest are 0s
positions = np.arange(1, 31)
data = np.zeros(30)
data[0] = 1  # 1st place

def update_chart(alpha=0.5, start_value=100, offset=0):
    """
    Updates and plots the scoring curve based on alpha, start_value, and offset.
    Also prints a two-column DataFrame of Place and Points.
    """
    # Get the exponentially smoothed distribution
    scoring_curve = exponential_smoothing(data, alpha, start_value)
    # Add the offset to each position in the curve
    final_curve = [pt + offset for pt in scoring_curve]
    
    # Clear current figure
    plt.figure(figsize=(5, 3))
    
    # Plot the scoring curve
    plt.plot(positions, final_curve, marker='o', label='Scoring Curve', color='blue')
    plt.plot(socring_curve_24['place'], socring_curve_24['points'], marker='o', label='2024 Scoring Curve', color='orange')
    plt.title(
        f'Fantasy League Scoring Curve\n'
        f'(1st place = {start_value+offset} pts, alpha = {alpha}, offset = {offset})'
    )
    plt.xlabel('Position')
    plt.ylabel('Points')
    plt.grid(True)
    plt.legend()
    
    # Freeze the y-axis range from 0 to 350
    plt.ylim(0, 350)
    
    plt.show()
    
    # Create a two-column DataFrame (Position, Points) and display it
    df = pd.DataFrame({
        'Position': positions,
        'Points': final_curve
    })
    # Optionally round points for a cleaner display
    df['Points'] = df['Points'].round()
    display(df.set_index('Position'))

    #Write to file
    df.to_csv('./data/points_map_2025.csv')

# Create interactive sliders for alpha, start_value, and offset
interactive_plot = interactive(
    update_chart,
    alpha=widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='Alpha'),
    start_value=widgets.FloatSlider(value=100, min=10, max=500, step=10, description='Start Value'),
    offset=widgets.FloatSlider(value=0, min=0, max=100, step=5, description='Offset')
)

# Display the interactive widget
display(interactive_plot)


interactive(children=(FloatSlider(value=0.5, description='Alpha', max=1.0, step=0.01), FloatSlider(value=100.0…

In [12]:
#Write scoring curve to
df

,place,points
0,1,300
1,2,245
2,3,195
3,4,155
4,5,135
5,6,110
6,7,97
7,8,85
8,9,75
9,10,64
